In [37]:
import os
import sys
import cv2
import torch
import mediapipe as mp
import numpy as np
import torch.nn as nn
from ultralytics import YOLO
from scipy.spatial import distance
from torch.nn.functional import softmax
from collections import defaultdict
from tqdm import tqdm

In [38]:
# YOLOv8s model 호출
yolo_model = YOLO('D:\\Falldown\\code-git\\runs\\detect\\human_fall_s30\\weights\\best.pt')

# MediaPipe Pose 호출
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, enable_segmentation=False)

# 랜드마크 인덱스 정의 # 11개
LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

# GRU 기반 낙상 감지 모델 정의
class FallDetectionGRU(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=2, num_classes=3):
        super(FallDetectionGRU, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = torch.zeros(2, x.size(0), 128).to(x.device)  # 초기 은닉 상태 정의
        out, _ = self.gru(x, h_0)
        out = self.fc(out[:, -1, :])  # 마지막 time step의 출력을 사용
        return out

def load_gru_model(model_path):
    # 모델을 생성할 때 올바른 input_size를 사용
    if "full" in model_path:
        input_size = 25
    elif "simplified" in model_path:
        input_size = 3
    elif "mediapipe" in model_path:
        input_size = 22
    else:
        raise ValueError("Unknown model type in filename. Please check the model name.")

    # FallDetectionGRU 모델 생성 및 가중치 로드
    gru_model = FallDetectionGRU(input_size=input_size, hidden_size=128, num_layers=2, num_classes=3)
    gru_model.load_state_dict(torch.load(model_path, map_location='cpu'))
    gru_model.eval()

    return gru_model, input_size

In [39]:
def bbox_ratio_class(ratio):
    return 0 if ratio < 0.7 else 1 # 0.7 이상은 Normal일 확률이 높음

def calculate_yolo_xy_ratio(bbox):
    # bbox가 비어 있지 않고, 폭(width)이 0이 아닌 경우에만 계산
    if len(bbox) > 0 and (bbox[2] - bbox[0]) != 0:
        return round((bbox[3] - bbox[1]) / (bbox[2] - bbox[0]), 3)
    return 0.0

# Head and upper body speed calculation
def calculate_head_upper_body_speed(sequence):
    speeds = []
    for j in range(1, len(sequence)):
        keypoints = sequence[j]
        prev_keypoints = sequence[j - 1]
        h = np.array([keypoints[0][0], keypoints[0][1]])  # Head
        l = np.array([keypoints[11][0], keypoints[11][1]])  # Left shoulder
        r = np.array([keypoints[12][0], keypoints[12][1]])  # Right shoulder

        prev_h = np.array([prev_keypoints[0][0], prev_keypoints[0][1]])
        prev_l = np.array([prev_keypoints[11][0], prev_keypoints[11][1]])
        prev_r = np.array([prev_keypoints[12][0], prev_keypoints[12][1]])

        center_new = (h + l + r) / 3
        center_prev = (prev_h + prev_l + prev_r) / 3

        speeds.append(distance.euclidean(center_new, center_prev))

    return sum(speeds) / len(speeds) if speeds else 0.0

In [40]:
# 바운딩 박스 크기 조절(20% 확대)
def adjust_bbox(bbox, scale_factor, frame_shape):
    x1, y1, x2, y2 = bbox
    width = x2 - x1
    height = y2 - y1
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    
    new_width = width * scale_factor
    new_height = height * scale_factor
    
    new_x1 = max(0, int(center_x - new_width / 2))
    new_y1 = max(0, int(center_y - new_height / 2))
    new_x2 = min(int(center_x + new_width / 2), frame_shape[1])  # 우측 하단 x 좌표를 조정, 이미지 폭(frame_shape[1]) 초과 방지
    new_y2 = min(int(center_y + new_height / 2), frame_shape[0])  # 우측 하단 y 좌표를 조정, 이미지 높이(frame_shape[0]) 초과 방지
    
    return [new_x1, new_y1, new_x2, new_y2]

In [41]:
# 샘플 Video를 분석하는 함수
def process_video(video_path, output_path, model_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    gru_model, input_size = load_gru_model(model_path)

    frame_idx = 0
    joint_sequence = []  # 속도 계산을 위함
    previous_pred_class = -1  # 이전 프레임의 예측 클래스

    # tqdm 초기화
    pbar = tqdm(total=total_frames, desc=f"Processing {os.path.basename(video_path)}", unit="frame", leave=True)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % 6 == 0:
            # YOLO inference
            results = yolo_model(frame)
            bbox = []

            # 바운딩 박스 추출 및 크기 조정
            for result in results[0].boxes:
                x1, y1, x2, y2 = map(int, result.xyxy[0])
                adjusted_bbox = adjust_bbox([x1, y1, x2, y2], scale_factor=1.2, frame_shape=frame.shape)
                bbox.append((adjusted_bbox[0], adjusted_bbox[1], adjusted_bbox[2], adjusted_bbox[3]))

            # MediaPipe inference
            joint_coords = []
            if len(bbox) > 0:
                # 조정된 바운딩 박스 사용
                person_image = frame[bbox[0][1]:bbox[0][3], bbox[0][0]:bbox[0][2]]
                person_image_rgb = cv2.cvtColor(person_image, cv2.COLOR_BGR2RGB)
                results_pose = pose.process(person_image_rgb)

                if results_pose.pose_landmarks:
                    for landmark in results_pose.pose_landmarks.landmark:
                        # 조정된 바운딩 박스 내의 좌표를 전체 프레임의 좌표로 변환
                        global_x = (bbox[0][0] + landmark.x * person_image.shape[1]) / width
                        global_y = (bbox[0][1] + landmark.y * person_image.shape[0]) / height
                        joint_coords.append((global_x * width, global_y * height))
                    joint_sequence.append(joint_coords)

            # Prepare input for GRU
            input_features = []

            if input_size == 25 and len(joint_coords) >= 11 and len(bbox) > 0:
                # 11개의 랜드마크 정보 추가 (각각 x, y 좌표를 사용)
                joint_array = []
                for landmark_idx in LANDMARKS:
                    if landmark_idx < len(joint_coords):
                        x, y = joint_coords[landmark_idx]
                        joint_array.extend([x, y])  # 각 랜드마크의 x, y 좌표 추가
                    else:
                        # 랜드마크가 존재하지 않는 경우, (0, 0)을 추가
                        joint_array.extend([0.0, 0.0])

                # 추가적인 특징 추가
                yolo_xy_ratio = calculate_yolo_xy_ratio(bbox[0])  # 바운딩 박스의 정보로부터 계산
                ratio_class = bbox_ratio_class(yolo_xy_ratio)
                speed = calculate_head_upper_body_speed(joint_sequence)

                # 모든 특징들을 하나의 배열로 결합 (총 25개가 되어야 함)
                input_features = np.array(joint_array)  # 총 22개의 관절 좌표 정보
                input_features = np.concatenate((input_features, [yolo_xy_ratio, ratio_class, speed]))  # 22 + 3 = 25개의 특징

                # 길이 확인
                if len(input_features) != 25:
                    raise ValueError(f"Expected input_features length to be 25, but got {len(input_features)}. Please check the feature extraction.")

            elif input_size == 22 and len(joint_coords) >= 11:
                # 11개의 랜드마크 좌표만 사용하는 경우 (각각 x, y 좌표)
                joint_array = []
                for landmark_idx in LANDMARKS:
                    if landmark_idx < len(joint_coords):
                        x, y = joint_coords[landmark_idx]
                        joint_array.extend([x, y])  # 각 랜드마크의 x, y 좌표 추가
                    else:
                        # 랜드마크가 누락된 경우, (0, 0) 추가
                        joint_array.extend([0.0, 0.0])
                input_features = np.array(joint_array)

                # 길이 확인
                if len(input_features) != 22:
                    raise ValueError(f"Expected input_features length to be 22, but got {len(input_features)}. Please check the joint coordinates.")

            elif input_size == 3 and len(bbox) > 0:
                # 간소화된 바운딩 박스 정보만 사용하는 경우
                yolo_xy_ratio = calculate_yolo_xy_ratio(bbox[0])
                ratio_class = bbox_ratio_class(yolo_xy_ratio)
                speed = calculate_head_upper_body_speed(joint_sequence)

                # 3개의 특징으로 구성
                input_features = np.array([yolo_xy_ratio, ratio_class, speed])

                # 길이 확인
                if len(input_features) != 3:
                    raise ValueError(f"Expected input_features length to be 3, but got {len(input_features)}. Please check the bbox features.")

            else:
                pred_class = previous_pred_class  # 감지되지 않는 경우 이전 예측 클래스를 유지
                input_features = None

            if input_features is not None:
                # 입력 텐서의 형태: [input_size] -> [1, 1, input_size]로 변환
                input_tensor = torch.tensor(input_features, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # [1, 1, input_size]

                # GRU 모델에 전달 (sequence_length=1, batch_size=1, input_size=25)
                output = gru_model(input_tensor)
                pred_class = torch.argmax(softmax(output, dim=1), dim=1).item()
                previous_pred_class = pred_class  # 이전 예측 클래스 업데이트
            else:
                pred_class = previous_pred_class

        # Visualization (클래스에 따른 색상 설정)
        color_map = {
            0: (0, 255, 0),      # Normal: 초록색
            1: (0, 165, 255),    # Danger: 주황색
            2: (0, 0, 255)       # Fall: 빨간색
        }
        label_map = {
            0: 'Normal',    # 비낙상
            1: 'Danger',    # 낙상 위험
            2: 'Fall'       # 완전 낙상
        }
        color = color_map.get(previous_pred_class, (255, 255, 255))  # 기본값은 흰색 (클래스가 없는 경우)
        label_text = label_map.get(previous_pred_class, 'Unknown')   # 기본값은 'Unknown' (클래스가 없는 경우)

        # 바운딩 박스 그리기
        for box in bbox:
            x1, y1, x2, y2 = box
            label = f"Class: {label_text}"  # 클래스 이름을 직접 출력
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness=3)  # 바운딩 박스 테두리 두께를 3으로 설정
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, thickness=3)  # 글자 크기 및 두께 설정

        out.write(frame) # frame 저장
        pbar.update(1) # tqdm 업데이트
        frame_idx += 1

    cap.release()
    out.release()

In [42]:
model_group = ['best_fall_detection_gru_001_full.pt']

# model_group = ['best_fall_detection_gru_001_full.pt', 'best_fall_detection_gru_001_mediapipe.pt',
#                'best_fall_detection_gru_0001_simplified.pt', 'best_fall_detection_cnn_mediapipe_0001.pt']

video_group = ['D:\\Falldown\\Dataset\\Video_Dataset\\Video\Test\\00028_H_A_FY_C1.mp4', #1
               'D:\\Falldown\\Dataset\\Video_Dataset\\Video\Test\\00130_H_A_FY_C2.mp4', #2
               'D:\\Falldown\\Dataset\\Video_Dataset\\Video\Test\\00712_H_D_BY_C3.mp4', #3
               'D:\\Falldown\\Dataset\\Video_Dataset\\Video\Test\\02900_Y_C_BY_C4.mp4', #4
               'D:\\Falldown\\Dataset\\Video_Dataset\\Video\Test\\02087_H_A_SY_C5.mp4', #5
               'D:\\Falldown\\Dataset\\Video_Dataset\\Video\Test\\01757_Y_E_SY_C6.mp4', #6
               'D:\\Falldown\\Dataset\\Video_Dataset\\Video\Test\\00690_H_D_N_C7.mp4', #7
               'D:\\Falldown\\Dataset\\Video_Dataset\\Video\Test\\00799_O_E_N_C8.mp4'  #8
]

output_directory = 'D:\\Falldown\\code-git'

# Video 샘플 처리
for model_path in model_group:
    for video_path in video_group:
        suffix = f"_{model_path.split('_')[3]}_{model_path.split('_')[5].replace('.pt', '')}.mp4" if '_gru_' in model_path else f"_{model_path.split('_')[3]}.mp4" # suffix를 사용하여 각 파일에 따라 구분되는 파일명 만들기
        video_name, _ = os.path.splitext(os.path.basename(video_path)) # 비디오 파일명 추출
        output_filename = f"{video_name}{suffix}"
        output_path = os.path.join(output_directory, output_filename)

        process_video(video_path, output_path, model_path) # 비디오 처리 함수 호출
        print(f"✅ {video_name} 처리 완료!")

Processing 00028_H_A_FY_C1.mp4:   0%|          | 0/600 [00:00<?, ?frame/s]


0: 384x640 1 Non_Fall, 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


Processing 00028_H_A_FY_C1.mp4:   1%|          | 4/600 [00:00<01:12,  8.18frame/s]


0: 384x640 1 Non_Fall, 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


Processing 00028_H_A_FY_C1.mp4:   2%|▏         | 10/600 [00:00<00:36, 16.17frame/s]

...


0: 384x640 1 Fall, 30.0ms
Speed: 2.0ms preprocess, 30.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


Processing 00028_H_A_FY_C1.mp4:  43%|████▎     | 257/600 [00:13<00:15, 22.25frame/s]


0: 384x640 1 Fall, 31.0ms
Speed: 2.0ms preprocess, 31.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)


Processing 00028_H_A_FY_C1.mp4:  44%|████▍     | 263/600 [00:13<00:15, 21.94frame/s]

KeyboardInterrupt: 